## Zv7yPNR5W2_Assignment2_NLP

### 1- Concatenate all the fourth file

In [1]:
#import library
import pandas as pd
import os
import re
import string
import numpy as np
from random import randint
import nltk
import contractions
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Embedding
from tensorflow.keras.models import load_model
from pickle import dump, load
import nltk
from nltk import ngrams
from nltk.translate.bleu_score import sentence_bleu
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

ModuleNotFoundError: No module named 'nltk'

In [ ]:
#!pip install contractions

### Upload all the four doc , clean them before concatenate into one file

In [212]:
PATH_BASE = "Data"

# Function to read a document from file
def read_document(filename):
    with open(filename, 'r', encoding='latin-1') as file:
        # Use latin-1 encoding because there are some symbols that utf-8 isn't able to read
        return file.read()

def clean_text(text):
  # Convert to lowercase
    text = text.lower()

     # Remove punctuation certain punstuation and digits
  # Define punctuation to be remove we can see that [.!?] are not include we are going to use this so split the sentence further
    punctuation = '"#$%&\'()*+/:;<=>@[\\]^_`{|};,'

    text = re.sub(f"[{punctuation}{string.digits}]", "", text)

     # Remove extra whitespaces
    text = re.sub(r"\s+", " ", text).strip()

    return text

def expand_contractions(text):
    words = text.split(" ")
    words_expand = []
    for word in words:
        if word in contractions.contractions_dict:
            words_expand.append(contractions.fix(word))
        else:
            words_expand.append(word)

    text_expand = ' '.join(words_expand)
    return text_expand

def test_clean(text):
    # Expand contractions
    expanded_text = expand_contractions(text)

    # Clean the text
    cleaned_text = clean_text(expanded_text)

    return cleaned_text

# File names
file_names = [
    'A Man of the People.txt',
    'Anthills of the Savannah.txt',
    'Arrow of God.txt',
    'Things Fall Apart.txt'
]

# Read and clean each document
docs = [test_clean(read_document(os.path.join(PATH_BASE, file_name))) for file_name in file_names]




# Concatenate the content of all cleaned documents
concatenated_text_data = '\n'.join(docs)

In [213]:
 #Display the head of the concatenated_text
head_text = concatenated_text_data[:500]  # Change 100 to the desired number of characters
print("Head of concatenated text:")
print(head_text)

# Display the end of the concatenated_text
end_text = concatenated_text_data[-500:]  # Change 100 to the desired number of characters
print("\nEnd of concatenated text:")
print(end_text)

Head of concatenated text:
chinua achebe a man of the people first published in for chris chapter one no one can deny that chief the honourable m.a. nanga m.p. was the most approachable politician in the country. whether you asked in the city or in his home village anata they would tell you he was a man of the people. i have to admit this from the onset or else the story i am going to tell will make no sense. that afternoon he was due to address the staff and students of the anata grammar school where i was teaching at th

End of concatenated text:
 states canada and nigeria. he is also the recipient of nigerias highest award for intellectual achievement the nigerian national merit award. at present mr. achebe lives with his wife in annandale new york where they both teach at bard college. they have four children. also by chinua achebe anthills of the savannah arrow of god girls at war and other stories a man of the people no longer at ease nonfiction hopes and impediments selected ess

We see that the beginning of the result file is the the head of first file and the end is the tail of the last file

## 2- split the data into train( train set content the validation set ) and test  

In [214]:
from sklearn.model_selection import train_test_split

In [215]:
# Split the concatenated text into sentences using multiple punctuation marks
sentences_by_punctuation = re.split(r'[.!?]', concatenated_text_data)
#print the 10 first sentences
sentences_by_punctuation[0:10]

['chinua achebe a man of the people first published in for chris chapter one no one can deny that chief the honourable m',
 'a',
 ' nanga m',
 'p',
 ' was the most approachable politician in the country',
 ' whether you asked in the city or in his home village anata they would tell you he was a man of the people',
 ' i have to admit this from the onset or else the story i am going to tell will make no sense',
 ' that afternoon he was due to address the staff and students of the anata grammar school where i was teaching at the time',
 ' but as usual in those highly political times the villagers moved in and virtually took over',
 ' the assembly hall must have carried well over thrice its capacity']

We can see that the sentence are very well define. Just at the beginning that we have a small error because the name of the author is write in the article with the dot. The idea behind operation of the entire data it to avoid during the splitting ( train , test and validation) the sentence that don't make sense.

In [217]:
# we split our data 70% for train 15% for test and 15% for validation
train_data, global_test_data = train_test_split(sentences_by_punctuation, test_size=0.30, random_state=42)

val_data, test_data = train_test_split(global_test_data, test_size=0.5, random_state=42)

# Let us join the texts to make a file as beginning with sentence
train_text = ' '.join(train_data)
val_text = ' '.join(val_data)
test_text = ' '.join(test_data)

# We save our data
with open('train_text.txt', 'w') as file:
    file.write(train_text)

with open('val_text.txt', 'w') as file:
    file.write(val_text)

with open('test_text.txt', 'w') as file:
    file.write(test_text)


In [218]:
# for each data we get the test file that we can print the first 100 caracter
train_text[:100]

' he sat with a short wooden-headed knife between two heaps of yams  he was still looking at his palm'

In [284]:
test_text[:100]

' but he was struck as most people were by okonkwos brusqueness in dealing with less successful men  '

In [219]:
from nltk import bigrams, trigrams
from collections import Counter, defaultdict
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
from nltk import word_tokenize, sent_tokenize
from nltk import ngrams
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/student23/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [220]:
# Tokenize the text.
tokenized_text = [list(map(str.lower, word_tokenize(sent)))
                  for sent in sent_tokenize(train_text)]

In [221]:
# we get a list of one element which is the list of the word
tokenized_text[0][10]

'of'

In [222]:
# Preprocess the tokenized text for 2-grams language modelling
n = 2 # n gram
train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)
# Convert to lists
train_data_list = list(train_data)

## Training an bi-gram Model

In [223]:
# initialize MLE(maximun likelihood) model - MLE model, creates an empty vocabulary
model = MLE(n)

In [224]:
# before the train the vocab is o
len(model.vocab)

0

In [225]:
# traine model
model.fit(train_data_list, padded_sents)
print(model.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 12754 items>


In [226]:
# after traitement the voab is different
len(model.vocab)

12754

In [227]:
# The set of word in our model
print(model.vocab.lookup(tokenized_text[0][0:20]))

('he', 'sat', 'with', 'a', 'short', 'wooden-headed', 'knife', 'between', 'two', 'heaps', 'of', 'yams', 'he', 'was', 'still', 'looking', 'at', 'his', 'palm', 'he')


In [228]:
len(model.vocab.lookup(tokenized_text[0]))

193802

In [229]:
# the model automatically replace words not in the vocabulary with `<UNK>`.
print(model.vocab.lookup('language is never random unseenword .'.split()))

('language', 'is', 'never', 'random', '<UNK>', '<UNK>')


## Explore our model

In [39]:
print(model.counts)

<NgramCounter with 2 ngram orders and 387607 ngrams>


Counts the number of bigram of our model.

In [40]:
# Evaluate perplexity
perplexity = model.perplexity("a")
print("Perplexity:", perplexity)

Perplexity: 50.00103199174407


In [41]:
# Evaluate perplexity
perplexity = model.perplexity("heaps")
print("Perplexity:", perplexity)

Perplexity: 4023.397757057321


We Notice the perplexity of common word is small and for the rare word is high. This can be explain because the perplexity is inverse proportionnal to the  probability (frequency)

In [42]:
# probability of 'has' given 'has'
model.score('he', 'has'.split())  # P('has'|'he') [he has]

0.02097902097902098

## Evaluate the perplexity of the test set with our bigram language model.

In [43]:
from nltk.lm.preprocessing import pad_both_ends

tokenized_test_text = [list(map(str.lower, word_tokenize(sent)))
                  for sent in sent_tokenize(train_text)]

test_ngrams = list(pad_both_ends(ngrams(tokenized_test_text[0], n),n))


# Compute perplexity for the test set
test_perplexity = model.perplexity(test_ngrams)
print("Test Perplexity:", test_perplexity)


Test Perplexity: inf


In [44]:
# A test gra is a list of two words
test_ngrams[1:4]

[('he', 'sat'), ('sat', 'with'), ('with', 'a')]

**Comment:** Our perplexity if infinity mean that this bigram model is very worse to predict on unseen data. So The probability of the test set is too small. This can be explain by the lack of data. The data is not enough to train very well the model to get good result for perplexity.

In [46]:
for ngram in test_ngrams[0:10]:
    prob = model.score(ngram[-1], ngram[:-1])
    if prob == 0 :
        print(f"Probability of {ngram}: {prob}")

Probability of <s>: 0


We investigate of there somme bigramm with zero probability.We realize that there is not bigram in our test set we zero probability.So here we don't need smothing method here.

In [435]:
for ngram in test_ngrams[0:10]:
    prob = model.score(ngram[-1], ngram[:-1])
    print(f"Probability of {ngram}: {prob}")

AttributeError: 'Sequential' object has no attribute 'score'

This previous code display the probability of 10 frist bigram. we can notice that must probability is 

## Generating text

In [47]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(model, num_words, random_seed=42):
    """Generate text method  """
    content = []
    for token in model.generate(num_words, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)


In [48]:
# with our model we try to generate a text for 23 word ramdomly.
generate_sent(model,23, random_seed=90)

'did he sat nearest to help look out their families let her people and sneered why there last time as the day or'

We try test generation many time and we realize that there word generate doesn't have reel semantic or any strong grammatical construction.

## Compute the Blue score
 Here we evaluate using the reference sentence and using the entire test set.

In [53]:
# Define a function to generate a sample sentence using the n-gram model
def generate_ngram_sentence(seed_word, n, model, max_length=20):
    sentence = [seed_word]
    for _ in range(max_length):
        next_word = model.generate()
        sentence.append(next_word)
    return sentence

# Generate a sample sentence using the trained model
generated_sentence = generate_ngram_sentence("he", n, model)
print("Generated Sentence:",generated_sentence)

# Define the reference sentence
reference_sentence = "This is a sample sentence."
reference_tokens = nltk.word_tokenize(reference_sentence.lower())

# Convert generated sentence to lowercase
generated_tokens = [str(token).lower() for token in generated_sentence]

# Calculate BLEU score for the generated sentence
bleu_score = sentence_bleu([reference_tokens], generated_tokens, smoothing_function=SmoothingFunction().method1)
print("BLEU Score:", bleu_score)

Generated Sentence: ['he', 'clarke', 'minister', 'seeing', 'them', 'that', 'or', 'the', 'knee', 'visits', 'great', 'animals', 'him', 'to', 'mean', 'sleepy', 'stop', 'a', 'of', 'on', 'to']
BLEU Score: 0.00913442366656447


In [195]:
# ## Function to generate a sentence using the n-gram model
def generate_ngram_sentence(seed_word, n, model, max_length=223471):
    sentence = [seed_word]
    for _ in range(max_length):
        next_word = model.generate()
        sentence.append(next_word)
    return sentence

# Use the test set for BLEU score calculation
#test_reference_sentence = "This is a test sentence."  # Replace with an actual sentence from your test set
test_reference_tokens = nltk.word_tokenize(test_text)

# Generate a sample sentence using the trained model
generated_sentence = generate_sent(model,20, random_seed=42)
#print("Generated Sentence:", generated_sentence)

# Calculate BLEU score for the generated sentence using the test reference
generated_tokens = [str(token) for token in generated_sentence]
bleu_score = sentence_bleu([test_reference_tokens], generated_tokens, smoothing_function=SmoothingFunction().method1)
print("BLEU Score:", bleu_score)

BLEU Score: 2.9945485131931132e-164


### comment 
The Blue score of bigram model is very worse so to to better performance we can use The neural network lamguage modeling that will try to implemnet at the next section

### Implement RNN model (one word in one word out)
Model properties:

- model has an embedding layer to learn the word embedding
- he input sequence contains a single word therefore input_length = 1
- the model has a single LSTM layer with 50 units
- output layer has a softmax activation function and is comprised of one neuron for each word in the vocabulary


In [568]:
# integer encode text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([train_text])
encoded = tokenizer.texts_to_sequences([train_text])[0]

In [569]:
# convert text to sequence for the test_text to evaluate the model
X_test_seq = tokenizer.texts_to_sequences(test_text)

In [570]:
X_test_seq = [num if sublist else 0 for sublist in X_test_seq for num in sublist]

In [571]:
len(X_test_seq)

150971

In [572]:
len(encoded)

194567

In [573]:
# determine the vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 12072


Create sequences of words to fit the model with one word as input and one word as output

In [574]:
# create word -> word sequences for the train
sequences = list()
for i in range(1, len(encoded)):
    sequence = encoded[i-1:i+1]
    sequences.append(sequence)

print('Total Sequences: %d' % len(sequences))

Total Sequences: 194566


In [575]:
# create word -> word sequences for the test
test_sequence = list()
for i in range(1, len(X_test_seq)):
    test_sequence1 = X_test_seq[i-1:i+1]
    test_sequence.append(test_sequence1)

print('Total Sequences: %d' % len(test_sequence))

Total Sequences: 150970


In [576]:
# pad input sequences for train
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)

Max Sequence Length: 2


In [577]:
# pad input sequences for test
# Pad sequences to the maximum length
max_length_test = max(len(seq) for seq in test_sequence)
padded_sequences_test = pad_sequences(test_sequence, maxlen=max_length_test, padding='pre')
print('Max Sequence Length: %d' % max_length_test)

Max Sequence Length: 2


In [578]:
# split sequence into input X and output y
sequences = np.array(sequences)
X, y = sequences[:,0], sequences[:,1]

In [579]:
# separate test into input and output

# Convert the sequences to a 2D numpy array
sequences_array_test = np.array(padded_sequences_test)

X_test, y_test = sequences_array_test[:, :-1], sequences_array_test[:, -1]

In [580]:
X_test = X_test.reshape((X_test.shape[0], 1))
y_test = y_test.reshape((y_test.shape[0], 1))

In [581]:
X_test.shape, y_test.shape

((150970, 1), (150970, 1))

In [582]:
X = X.reshape((X.shape[0], 1))
y = y.reshape((y.shape[0], 1))

In [583]:
X.shape, y.shape

((194566, 1), (194566, 1))

To fit a model to predict a probability distribution across all words in the vocabulary. We need to turn the output element (y) from a single integer into a one hot encoding with a 0 for every word in the vocabulary and a 1 for the actual word.

In [584]:
# one hot encode outputs
y = to_categorical(y, num_classes=vocab_size)

In [585]:
# One-hot encode the output variable of the test
y_test = to_categorical(y_test, num_classes=vocab_size)

In [586]:
y_test.shape

(150970, 12072)

### Train neural language model (one word in one word out)

In [415]:
# define the model
def define_model(vocab_size):
    """Define the Deep learning """
    model = Sequential()
    model.add(Embedding(vocab_size, 10))
    model.add(LSTM(50, input_shape=(1, 10), return_sequences=True)) 
    model.add(Dense(vocab_size, activation='softmax'))

    # compile network
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    plot_model(model, to_file='model.png', show_shapes=True)

    return model

In [416]:
def define_model(vocab_size):
    """Define the Deep learning """
    model = Sequential()
    model.add(Embedding(vocab_size, 10))  # Remove input_length argument
    model.add(LSTM(50))
    model.add(Dense(vocab_size, activation='softmax'))
    
    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # Print model summary
    model.summary()
    
    # Build the model
    model.build((None,10))  # Input shape with None indicates variable sequence length
    
    # Plot the model
    plot_model(model, to_file='model.png', show_shapes=True)

    return model

In [417]:
# define model
model = define_model(vocab_size)

model.fit(X, y, epochs=10, verbose=1)

Model: "sequential_24"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_24 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_22 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.
Epoch 1/10
6081/6081 ━━━━━━━━━━━━━━━━━━━━ 75s 12ms/step - accuracy: 0.0540 - loss: 7.0675
Epoch 2/10
6081/6081 ━━━━━━━━━━━━━━━━━━━━ 71s 12ms/step - accuracy: 0.0748 - loss: 6.2620
Epoch 3/10
6081/6081 ━━━━━━━━━━━━━━━━━━━━ 75s 12ms/step - accuracy: 0.1040 - loss: 5.9341
Epoch 4/10
6081/6081 ━━━━━━━━━━━━━━━━━━━━ 79s 13ms/step - accuracy: 0.1145 - loss: 5.7703
Epoch 5/10
6081/6081 ━━━━━━━━━━━━━━━━━━━━ 80s 13ms/step - accuracy: 0.1226 - loss: 5.6490
Epoch 6/10
6081/6081 ━━━━━━━━━━━━━━━━━━━━ 76s 12ms/step - accuracy: 0.1277 - loss: 5.5564
Epoch 7/10
6081/6081 ━━━━━━━━━━━━━━━━━━━━ 75s 12ms/step - accuracy: 0.1322 - loss: 5.4930
Epoch 8/10
6081/6081 ━━━━━━━━━━━━━━━━━━━━ 73s 12ms/step - accuracy: 0.1377 - loss: 5.4253
Epoch 9/10
6081/6081 ━━━━━━━━━━━━━━━━━━━━ 71s 12ms/step - accuracy: 0.1419 - loss: 5.3685
Epoch 10/10
6081/6081 ━━━━━━━━━━━━━━━━━━━━ 74s 12ms/step - accuracy: 0.1470 - l

The accuracy of our model increase as epoch increase and the loss decrease. we can say that our model learn well and will be better with the high number of epoch.

In [ ]:
from IPython.display import Image

Image(filename='model.png')


We realise that the table show the summary of the architecture of our model is not display (somme package needed) so we get the model image usin Golab(The model isn't able to train in Colab but we can see this table clearly)

## Generate text

In [418]:

# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer and truncate sequences to a fixed length
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        yhat = np.argmax(model.predict(encoded), axis=-1)

        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break

        # append to input
        in_text += ' ' + out_word
        result.append(out_word)

    return ' '.join(result)

In [419]:
sequence = re.split(r'[.!?]', concatenated_text_data)
seq_length = len(sequence[0].split()) - 1

In [420]:
sequence[0]

'chinua achebe a man of the people first published in for chris chapter one no one can deny that chief the honourable m'

In [421]:
seq_length

22

In [422]:
# select a seed text
idx = randint(0,len(sequence))# take random line in the full concat file
seed_text = sequence[idx]
print(seed_text + '\n')

 in the morning as there was no sign of life in her hut her husband pushed open the door and peeped in



In [423]:
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 10)
print('generated text')
print(generated)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 547ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
generated text
the sleazy under the odorous the other other people people


### Comment
The sequence generate by our Language Modeling is a little bit better than the bigram. And this can be better improve by parameter tuning.

## Evaluate the model 

### Compute the Perplexity of our model

In [424]:
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)

4718/4718 ━━━━━━━━━━━━━━━━━━━━ 31s 6ms/step - accuracy: 0.0387 - loss: 9.0948


In [431]:
print("The Loss of our test set is: ", test_loss)

The Loss of our test set is:  9.095501899719238


In [430]:
print("The accuracy of our test set is: ", test_accuracy)

The accuracy of our test set is:  0.03832549601793289


In [427]:
# Calculate perplexity 
perplexity = 2 ** test_loss

In [432]:
print("The perplexity of our test set is: ", perplexity)

The perplexity of our test set is:  547.0397656902501


### Comment 
We notice the complexity is high so the model doesn't perform very well. Here we can see the with more epoch we will get better performance but due to the lack of computer ressources we don't train it with a large epoch. 

### Compute the Bleue of our model

In [444]:
#upload the english word to be sure that the word generate exit in english vocabulary
Word_Alpha = pd.read_csv("words_alpha.txt",header= None, delimiter = "\t")
Word_Alpha.head()

,0
0,a
1,aa
2,aaa
3,aah
4,aahed


In [445]:
#Transform the word alphabet to list
english_words = Word_Alpha.values.tolist()

In [503]:
# Function to generate a sequence given a seed text
def generate_sequence(seed_text, model, max_length, tokenizer):
    """
    Generate a sequence of words using a trained LSTM model.

    Parameters:
    - seed_text (str): The initial text to start the sequence.
    - model (Sequential): The trained LSTM model.
    - max_length (int): The maximum length of the generated sequence.
    - tokenizer (Tokenizer): Tokenizer used for text encoding.
    - english_words (set): A set of English words for validation.

    Returns:
    - generated_sequence (str): The generated sequence of words.
    """
    generated_sequence = seed_text
    
    for _ in range(max_length):
        encoded_seed = tokenizer.texts_to_sequences([seed_text])[0]
        padded_seed = pad_sequences([encoded_seed], maxlen=max_length-1, padding='pre')
        
        # Use predict to get the probabilities of each word
        predicted_probs = model.predict(padded_seed, verbose=0)
        
        # Choose the word with the highest probability
        predicted_word_index = np.argmax(predicted_probs)
        predicted_word = tokenizer.index_word[predicted_word_index]
        
        seed_text += ' ' + predicted_word
        generated_sequence += ' ' + predicted_word
    
    return generated_sequence# in english_words


# Sample seed text
# select a seed text
idx = randint(0,len(sequence))# take random line in the full concat file
seed_text = sequence[idx]
print("The seed_test is : "+ seed_text + '\n')

# Generate a sequence using the trained LSTM model
max_length = 10  # Adjust this based on the maximum length of sequences in your training data
generated_sequence = generate_sequence(seed_text, model, max_length, tokenizer)#,english_words)

# Sample reference sentence for BLEU score calculation
reference_sentence = "The quick brown fox jumps over the lazy dog"

# Tokenize the reference sentence and the generated sequence
reference_tokens = nltk.word_tokenize(reference_sentence)
generated_tokens = nltk.word_tokenize(generated_sequence)

# Calculate BLEU score
bleu_score = sentence_bleu([reference_tokens], generated_tokens, smoothing_function=SmoothingFunction().method1)
print("Generate sequence is :",generated_sequence)
print("BLEU Score:", bleu_score)


The seed_test is : 

Generate sequence is :  and and the sleazy searching with the odorous the other
BLEU Score: 0.021105340631872645


### Comment 
The Blue score near to 1 mean that the model perform well. The Blue score here is far from 1 and near to zero so the model doesn't have the good performance. To enhance the performance of our model we need to add the epoch.

## Language model multiple input

Here the input sequence has high len than previous(which was just 2)

In [504]:
# organize into sequences of tokens
length = 50 +1
sequences = list()
# we use the token of the tokenized_text previous in ngram model represented the tokens of the train set 
for i in range(length, len(tokenized_text[0])):
    # select sequence of tokens
    seq = tokenized_text[0][i-length:i]
    # convert into a line
    line = ' '.join(seq)
    sequences.append(line)

print('Total Sequences: %d' % len(sequences))
print(len(seq))

Total Sequences: 193751
51


So we use the tokens of our train to generate 193751 sequences of 51 words.

In [510]:
# A sequence here is the chunck of 50 words
sequences[0:5]

['he sat with a short wooden-headed knife between two heaps of yams he was still looking at his palm he knew that he was a fierce fighter but that year-had been enough to break the heart of a lion but although he thought for a long time he found no answer',
 'sat with a short wooden-headed knife between two heaps of yams he was still looking at his palm he knew that he was a fierce fighter but that year-had been enough to break the heart of a lion but although he thought for a long time he found no answer eei',
 'with a short wooden-headed knife between two heaps of yams he was still looking at his palm he knew that he was a fierce fighter but that year-had been enough to break the heart of a lion but although he thought for a long time he found no answer eei what',
 'a short wooden-headed knife between two heaps of yams he was still looking at his palm he knew that he was a fierce fighter but that year-had been enough to break the heart of a lion but although he thought for a long tim

In [508]:
# our corpus of train has 1025656 tokens and we make a sequence of 50 words
len(tokenized_text[0])

193802

In [509]:
# integer encode sequences of words
tokenizer_M = Tokenizer()
tokenizer_M.fit_on_texts(sequences)
sequences_token = tokenizer_M.texts_to_sequences(sequences)

In [590]:
# vocabulary size
vocab_size2 = len(tokenizer_M.word_index) + 2 #to be the samelength as the previous
print(vocab_size2)

12072


We just notice that the vocabulary is almost the same as previous for one input.

In [591]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

# Pad sequences to the maximum length
max_length = max(len(seq) for seq in sequences_token)
padded_sequences = pad_sequences(sequences_token, maxlen=max_length, padding='pre')

# Convert the sequences to a 2D numpy array
sequences_array = np.array(padded_sequences)

# separate into input and output
X2, y2 = sequences_array[:, :-1], sequences_array[:, -1]

# One-hot encode the output variable
y2 = to_categorical(y2, num_classes=vocab_size2)

print('X2 Shape:', X2.shape)
print('y2 Shape:', y2.shape)


X2 Shape: (193751, 61)
y2 Shape: (193751, 12072)


To apply LSTM we need to the input to be in the same shape , that why we add padding in each input sequence.

In [592]:
def define_model(vocab_size):
    """Define the Deep learning """
    model2 = Sequential()
    model2.add(Embedding(vocab_size, 50))
    model2.add(LSTM(100, return_sequences=True))
    model2.add(LSTM(100))
    model2.add(Dense(100, activation='relu'))
    model2.add(Dense(vocab_size, activation='softmax'))
    
    # Compile the model
    model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # Print model summary
    model2.summary()
    
    # Build the model
    model2.build((None,61))  # Input shape with None indicates variable sequence length
    
    # Plot the model
    plot_model(model2, to_file='model.png', show_shapes=True)

    return model2

In [514]:
# define model
model2 = define_model(vocab_size2)

model2.fit(X2, y2, batch_size=32, epochs=5)

Model: "sequential_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_25 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_23 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_24 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.
Epoch 1/5
6055/6055 ━━━━━━━━━━━━━━━━━━━━ 496s 81ms/step - accuracy: 0.0591 - loss: 6.7258
Epoch 2/5
6055/6055 ━━━━━━━━━━━━━━━━━━━━ 533s 86ms/step - accuracy: 0.0934 - loss: 6.0576
Epoch 3/5
6055/6055 ━━━━━━━━━━━━━━━━━━━━ 529s 87ms/step - accuracy: 0.1082 - loss: 5.8237
Epoch 4/5
6055/6055 ━━━━━━━━━━━━━━━━━━━━ 532s 88ms/step - accuracy: 0.1167 - loss: 5.6341
Epoch 5/5
6055/6055 ━━━━━━━━━━━━━━━━━━━━ 555s 87ms/step - accuracy: 0.1223 - loss: 5.5002


We need to add epoch to better performance and also computer ressources. at 5 epoch we get 12% of accuracy. 

## Generate the test with this multiple output
 We use the function that we write before to generate the test.

In [516]:
sequence = re.split(r'[.!?]', concatenated_text_data)
seq_length = len(sequence[0].split()) - 1

In [532]:
# select a seed text
idx = randint(0,len(sequence))# take random line in the full concat file
seed_text = sequence[idx]
print(seed_text + '\n')

 they made single mounds of earth in straight lines all over the field and sowed the yams in them



In [533]:
# generate new text
generated = generate_seq(model2, tokenizer_M, seq_length, seed_text, 13)
print('generated text')
print(generated)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
generated text
and the man of the man of the man of the man of


### Comment: 
Our Model generate the word or sequence that not really make sens( grammatically not real or correct). This can be improve by train the model in more epoch, here we just try 2 epoch because we was't have enough capacity of RAM to run and train the model well.He can remarq how far the accuracy increase in the second epoch so with many epoch we can get good performance.

## Evaluation du model 

### Compute the complexity

In [589]:
# Evaluate on test set
test_loss2, test_accuracy2 = model2.evaluate(X_test, y_test)

In [ ]:
print("The Loss of our test set is: ", test_loss2)

In [ ]:
print("The accuracy of our test set is: ", test_accuracy2)

In [ ]:
# Calculate perplexity 
perplexity2 = 2 ** test_loss2

In [ ]:
print("The perplexity of our test set is: ", perplexity2)

## Compute the Blue Score

In [ ]:
# Function to generate a sequence given a seed text
def generate_sequence(seed_text, model, max_length, tokenizer):
    """
    Generate a sequence of words using a trained LSTM model.

    Parameters:
    - seed_text (str): The initial text to start the sequence.
    - model (Sequential): The trained LSTM model.
    - max_length (int): The maximum length of the generated sequence.
    - tokenizer (Tokenizer): Tokenizer used for text encoding.
    - english_words (set): A set of English words for validation.

    Returns:
    - generated_sequence (str): The generated sequence of words.
    """
    generated_sequence = seed_text
    
    for _ in range(max_length):
        encoded_seed = tokenizer.texts_to_sequences([seed_text])[0]
        padded_seed = pad_sequences([encoded_seed], maxlen=max_length-1, padding='pre')
        
        # Use predict to get the probabilities of each word
        predicted_probs = model.predict(padded_seed, verbose=0)
        
        # Choose the word with the highest probability
        predicted_word_index = np.argmax(predicted_probs)
        predicted_word = tokenizer.index_word[predicted_word_index]
        
        seed_text += ' ' + predicted_word
        generated_sequence += ' ' + predicted_word
    
    return generated_sequence# in english_words


# Sample seed text
# select a seed text
idx = randint(0,len(sequence))# take random line in the full concat file
seed_text = sequence[idx]
print("The seed_test is : "+ seed_text + '\n')

# Generate a sequence using the trained LSTM model
max_length = 10  # Adjust this based on the maximum length of sequences in your training data
generated_sequence = generate_sequence(seed_text, model2, max_length, tokenizer)#,english_words)

# Sample reference sentence for BLEU score calculation
reference_sentence = "The quick brown fox jumps over the lazy dog"

# Tokenize the reference sentence and the generated sequence
reference_tokens = nltk.word_tokenize(reference_sentence)
generated_tokens = nltk.word_tokenize(generated_sequence)

# Calculate BLEU score
bleu_score = sentence_bleu([reference_tokens], generated_tokens, smoothing_function=SmoothingFunction().method1)
print("Generate sequence is :",generated_sequence)
print("BLEU Score:", bleu_score)


## Conclusion

Summary, given 4 txt data, we concat those file, then split then into test, validation and test after some preprocessing(remove unseless punctuation, space, normalize this). We applied 3 several laguage model bigram (n=2), LSTM for one input, LSTM for multiple input.